In [ ]:
#pip install recommenders


In [ ]:
#print(tf.executing_eagerly())  # should be True now

In [ ]:
import kagglehub 
import pandas as pd 
import numpy as np 
import os 
import seaborn as sns
import random 
import matplotlib.pyplot as plt
import nltk
import ast
import re
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout,Attention,Multiply,GlobalAveragePooling1D
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from math import sqrt
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Lambda
from tensorflow.keras.optimizers import Adam
from time import time
#from recommenders.evaluation.python_evaluation import (
    #map, ndcg_at_k, precision_at_k, recall_at_k
#)
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm 
from collections import defaultdict,Counter

In [ ]:
print(tf.__version__)

#### STEP 1 : LOADING THE DATASET FROM KAGGLE 

In [ ]:
devices = tf.config.get_visible_devices()
print(devices)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
#path = kagglehub.dataset_download("samlearner/letterboxd-movie-ratings-data")

#print("Path to dataset files:", path)

In [ ]:
#movie_data_filename = 'movie_data.csv'
#ratings_filename = 'ratings_export.csv'
#users_filename = 'users_export.csv'
#movie_data_path = os.path.join(path,movie_data_filename)
#ratings_path = os.path.join(path,ratings_filename)
#users_path = os.path.join(path,users_filename)

In [ ]:
# movie_data_df = pd.read_csv(movie_data_path,engine='python')
# ratings_df = pd.read_csv(ratings_path,engine='python')
# users_df = pd.read_csv(users_path,engine = 'python')

In [ ]:
movie_data_df = pd.read_parquet('/kaggle/input/parquet-dataset/movie_data.parquet',engine='pyarrow')
ratings_df = pd.read_parquet('/kaggle/input/parquet-dataset/ratings_data.parquet',engine='pyarrow')
users_df = pd.read_parquet('/kaggle/input/parquet-dataset/users_dt.parquet',engine='pyarrow')

In [ ]:
def parse_genre_string(genre_str):
    # If the value is not a string (it could be float), convert to '[]'
    if not isinstance(genre_str, str):
        genre_str = '[]'
    # Now attempt parsing
    try:
        parsed = ast.literal_eval(genre_str)
        # If parsing somehow doesn't return a list, force it
        if not isinstance(parsed, list):
            return []
        return parsed
    except Exception:
        return []

In [ ]:
movie_data_df['genres'] = movie_data_df['genres'].fillna('[]')
movie_data_df['genres'] = movie_data_df['genres'].apply(parse_genre_string)

In [ ]:
#print(movie_data_df['genres'].head(50))
#print(movie_data_df['genres'].tail(50))
#print(movie_data_df['genres'].dtype)

In [ ]:
all_genres1 = set()
for g_list in movie_data_df['genres']:
    all_genres1.update(g_list)

print("Unique genres found:", all_genres1)
print("Number of unique genres:", len(all_genres1))

In [ ]:
print(users_df.head())


# this dataframe won't be used, as we don't want the models predictions to depend on the users data. 

unneccessary columns need to be removed - image url, imdb id, imdb, link (perhaps needed to enhance model with imdb data ? ), tmbd id, link, (download that dataset and enhance with it ? )production countries, 

In [ ]:
print(movie_data_df.info())


# we can check if adding additional information from this dataframe to the model improves its performance (genre of the movie, overview? tokenized, year_released)

In [ ]:
print(ratings_df.head())

In [ ]:
print(ratings_df.info())

In [ ]:
#movie_data_df.to_parquet('movie_data.parquet', engine='pyarrow')
#ratings_df.to_parquet('ratings_data.parquet', engine = 'pyarrow')
#users_df.to_parquet('users_dt.parquet',engine = 'pyarrow')
# Load from Parquet file



#### STEP 2: EXPLORATORY DATA ANALYSIS 

In [ ]:
movie_data_df = movie_data_df.dropna(subset=['overview'])

In [ ]:
movie_data_df['overview_length'] = movie_data_df['overview'].apply(len)

avg_length = movie_data_df['overview_length'].mean()
min_length = movie_data_df['overview_length'].min()
max_length = movie_data_df['overview_length'].max()
movie_data_df.drop(['overview_length'],axis = 1)
print(f"Average Overview Length: {avg_length:.2f}")
print(f"Minimum Overview Length: {min_length}")
print(f"Maximum Overview Length: {max_length}")


In [ ]:
ratings_df.dropna(subset=['user_id', 'movie_id', 'rating_val'], inplace=True)
movie_data_df.dropna(subset=['genres'], inplace=True)

In [ ]:
ratings_df.head()

In [ ]:
user_mapping_movie_data = {user : idx for idx, user in enumerate(movie_data_df['movie_id'].unique())}
user_mapping = {user: idx for idx, user in enumerate(ratings_df['user_id'].unique())}
item_mapping = {item: idx for idx, item in enumerate(ratings_df['movie_id'].unique())}
all_genres = set(genre for genres in movie_data_df['genres'] for genre in genres)
genre_mapping = {genre: idx for idx, genre in enumerate(sorted(all_genres))}
movie_data_df['genre_mapped'] = movie_data_df['genres'].apply(
     lambda genres: [genre_mapping[genre] for genre in genres if genre in genre_mapping]
 )
print(genre_mapping)

In [ ]:
sample_items = list(user_mapping.items())[:10]
print(sample_items)

In [ ]:
sample_items1 = list(user_mapping_movie_data.items())[:10]
print(sample_items1)

In [ ]:
movie_data_df['movie_id'] = movie_data_df['movie_id'].map(user_mapping_movie_data)
print(movie_data_df.head())

In [ ]:
ratings_df['user_id'] = ratings_df['user_id'].map(user_mapping)
ratings_df['movie_id'] = ratings_df['movie_id'].map(item_mapping)
ratings_df.head()

### non integer frames must be mapped to a unique numeric value - movie id and user id in this case

In [ ]:
ratings_df = ratings_df.drop(['_id'],axis = 1)
ratings_df.head()

In [ ]:
n_users = len(user_mapping)
n_items = len(item_mapping)
print(f'No. users : {n_users}, no. items : {n_items}')
#print(user_mapping)

In [ ]:
ratings_df['rating_val'] = ratings_df['rating_val'].astype(int)  

# Count the number of ratings per movie
rating_counts = ratings_df.groupby('movie_id')['rating_val'].count()

# Plot distribution
plt.figure(figsize=(12, 6))
plt.hist(rating_counts, bins=50, edgecolor='black', alpha=0.7)

# Labels and title
plt.xlabel("Number of Ratings per Movie")
plt.ylabel("Count of Movies")
plt.title("Distribution of Number of Ratings per Movie")
plt.yscale('log') 
plt.show()
plt.savefig('rating_freq.png')

In [ ]:
ratings_agg = ratings_df.groupby('movie_id').agg(
    avg_rating=('rating_val', 'mean'),
    num_ratings=('rating_val', 'count')
).reset_index()

movie_data_df['movie_id'] = movie_data_df['movie_id'].astype(int)  
merged_df = pd.merge(movie_data_df, ratings_agg, on='movie_id', how='inner')

# Optional: Convert popularity to numeric if necessary
merged_df['popularity'] = pd.to_numeric(merged_df['popularity'], errors='coerce')

# Step 3: Create the scatter plot
plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    merged_df['popularity'],
    merged_df['avg_rating'],
    s=merged_df['num_ratings'] * 0.1, 
    c=merged_df['vote_average'],       
    cmap='viridis',
    alpha=0.7
)
plt.xlabel('Popularity')
plt.ylabel('Average Rating')
plt.title('Relationship between Movie Popularity and Average Rating')
plt.colorbar(scatter, label='Vote Average')
plt.xscale('log') 
plt.grid(True)
plt.savefig('popularity_vs_rating.png')
plt.show()

#### STEP 3: DEFINING THE TASK, EVALUATION METRICS, CONSTRUCTING MODEL 

In [ ]:
#ncf_model.load_weights('ncf_model.weights.h5')
# model can be saved locally after training so it doesn't have to be redone again 

In [ ]:
train, val = train_test_split(ratings_df, test_size=0.2, random_state=42)
train, test = train_test_split(train, test_size = 0.2, random_state = 42)

Pointwise loss prefered in ranking systems. Need to construct triple of user,pos item, neg item, and train to model to successfully rate the pos item as high as possible. (previosly was done with pointwise loss and biased towards best rated movies ). 

In [ ]:
print(ratings_df['movie_id'].unique())

In [ ]:
print(ratings_df.size)

In [ ]:
threshold = 8  
num_items = 286069  # total num of movies 

In [ ]:
# Generate training triples: (user, pos_item, neg_item)
def generate_training_triples(ratings_df, num_items, threshold,user_pos):
    user_list, pos_item_list, neg_item_list = [], [], []
    
    for user, pos_items in tqdm(user_pos.items(), total=len(user_pos), desc="Processing users"):
        for pos_item in pos_items:
            # Sample a negative item
            # We'll sample until we find an item the user hasn't interacted with positively.
            while True:
                neg_item = np.random.randint(0, num_items)
                if neg_item not in pos_items:
                    break
            user_list.append(user)
            pos_item_list.append(pos_item)
            neg_item_list.append(neg_item)
            
    return np.array(user_list), np.array(pos_item_list), np.array(neg_item_list)


In [ ]:
positive_df = ratings_df['rating_val']

In [ ]:
user_pos = ratings_df.groupby('user_id')['movie_id'].agg(set).to_dict()

In [ ]:
users_all, pos_items_all, neg_items_all = generate_training_triples(ratings_df,num_items,threshold,user_pos)

In [ ]:
users_train, users_temp, pos_items_train, pos_items_temp, neg_items_train, neg_items_temp = train_test_split(
    users_all, pos_items_all, neg_items_all,
    test_size=0.4,  # 40% goes to temp (validation+test)
    random_state=42
)

# Then, split the temporary set equally into validation and test sets
users_val, users_test, pos_items_val, pos_items_test, neg_items_val, neg_items_test = train_test_split(
    users_temp, pos_items_temp, neg_items_temp,
    test_size=0.5,  # 50% of temp for validation, 50% for test (i.e., 20% of original each)
    random_state=42
)

In [ ]:
print(user_train)

In [ ]:
print(neg_item_train.size)

# easiest to construct model using functional API (multiple input network). 3 dense layers of sizes 64,32,16 respectively follow the embedding layers

### Pairwise loss model - bayesian personalized ranking loss function as per the paper BPR: Bayesian Personalized Ranking from Implicit Feedback

In [ ]:
embedding_dim = 32  # Latent factor size
mlp_layer_sizes = [64, 32, 16]  # Fully connected layers

In [ ]:
def bpr_loss(_, y_pred):
    pos_score, neg_score = y_pred[:, 0], y_pred[:, 1]
    loss = -K.mean(K.log(K.sigmoid(pos_score - neg_score)))
    return loss

In [ ]:
# user_train, pos_item_train, neg_item_train define needed 

In [ ]:
num_users = 7477
embedding_dim = 32
mlp_layer_sizes = [64, 32, 16, 8]

user_embedding_layer = Embedding(num_users, embedding_dim, name='user_embedding')
item_embedding_layer = Embedding(num_items, embedding_dim, name='item_embedding')

def scoring_model():
    user_in = Input(shape=(1,), name='user_in')
    item_in = Input(shape=(1,), name='item_in')
    
    user_vec = Flatten()(user_embedding_layer(user_in))
    item_vec = Flatten()(item_embedding_layer(item_in))
    
    gmf_vec = Multiply()([user_vec, item_vec])
    
    concat_vec = Concatenate()([user_vec, item_vec])
    mlp = concat_vec
    for size in mlp_layer_sizes:
        mlp = Dense(size, activation='relu')(mlp)
        mlp = Dropout(0.2)(mlp)
    
    pre_output = Concatenate()([gmf_vec, mlp])
    output = Dense(1, activation='linear')(pre_output)
    
    return Model(inputs=[user_in, item_in], outputs=output, name='scoring_model')

scoring_net = scoring_model()
scoring_net.summary()

user_input = Input(shape=(1,), name='user_input')
pos_item_input = Input(shape=(1,), name='pos_item_input')
neg_item_input = Input(shape=(1,), name='neg_item_input')

pos_score = scoring_net([user_input, pos_item_input])
neg_score = scoring_net([user_input, neg_item_input])

y_pred = Concatenate(axis=1, name='y_pred')([pos_score, neg_score])

pairwise_model = Model(inputs=[user_input, pos_item_input, neg_item_input], outputs=y_pred)
pairwise_model.summary()

def bpr_loss(_, y_pred):
    pos_score = y_pred[:, 0]
    neg_score = y_pred[:, 1]
    loss = -K.mean(K.log(K.sigmoid(pos_score - neg_score)))
    return loss

pairwise_model.compile(optimizer='adam', loss=bpr_loss)

dummy_y = np.zeros((neg_items_train.size, 2))
dummy_y_val = np.zeros((neg_items_val.size,2))
#pairwise_model.load_weights('/kaggle/input/pairwise_weights/tensorflow2/v1/1/pairwise_model_all.weights(1).h5')
pairwise_model.fit([users_train, pos_items_train, neg_items_train],
        dummy_y,
        validation_data=(
        [users_val, pos_items_val, neg_items_val],    
            dummy_y_val
    ),  epochs=3, batch_size=32)


In [ ]:
pairwise_model.save_weights('pairwise_model_all.weights.h5')

In [ ]:
print(pos_items_test[:5])

In [ ]:
print("pos_items_test shape:", pos_items_test.shape)
print("neg_items_test shape:", neg_items_test.shape)

In [ ]:
all_items_set = set(ratings_df['movie_id'].unique())

In [ ]:
train_dict = defaultdict(set)

# Loop over the training examples to build the dictionary
for user, item in zip(users_train, pos_items_train):
    train_dict[user].add(item)

# If you want a regular dictionary (instead of defaultdict), convert it:
train_dict = dict(train_dict)

In [ ]:
unique_user_ids = ratings_df['user_id'].unique()


k = 10
# Lists for evaluation metrics
hits = []
ndcgs = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

for user_id in tqdm(unique_user_ids, desc="Evaluating users"):
    # Get positive items for this user (from your pos arrays)
    pos_indices = np.where(users_test == user_id)[0]
    user_pos_items = pos_items_test[pos_indices]
    
    # Generate negative samples for the user:
    # Here we sample negatives from items that are NOT in the training set for the user.
    # You can adjust the number of negatives per positive as needed.
    n_neg_samples = 99
    negatives = []
    # Ensure that we sample enough negatives
    possible_negatives = list(all_items_set - train_dict.get(user_id, set()))
    if len(possible_negatives) < n_neg_samples:
        negatives = possible_negatives
    else:
        negatives = random.sample(possible_negatives, n_neg_samples)
    
    # For each positive item, create a test batch with the positive and the negative items.
    # Alternatively, if you want to evaluate the user as a whole, you could combine all positive items and negatives.
    # Here we demonstrate combining them for a single evaluation round.
    test_items = np.concatenate([user_pos_items, negatives])
    # Create an array of the same user id for each item
    user_batch = np.full(test_items.shape, user_id)
    
    # Predict scores using your scoring model (which outputs one score per (user, item) pair)
    scores = scoring_net.predict([user_batch, test_items], verbose=0)
    scores = np.squeeze(scores)  # Ensure it's 1D
    
    # Create labels: 1 for each positive, 0 for each negative.
    labels = np.concatenate([np.ones(len(user_pos_items)), np.zeros(len(negatives))])
    
    # For overall AUC
    all_true_labels.extend(labels)
    all_pred_scores.extend(scores)
    
    # Rank items based on scores
    sorted_indices = np.argsort(scores)[::-1]
    sorted_labels = labels[sorted_indices]
    sorted_items = test_items[sorted_indices]
    
    # Record top-k items for analysis
    top_k_items = sorted_items[:k]
    recommended_items.extend(top_k_items.tolist())
    
    # Compute Hit@k: Check if any positive is in the top k
    if 1 in sorted_labels[:k]:
        hits.append(1)
    else:
        hits.append(0)
    
    # Compute NDCG@k: Discounted gain for the first positive
    pos_ranks = np.where(sorted_labels[:k] == 1)[0]
    if pos_ranks.size > 0:
        ndcgs.append(1 / math.log2(pos_ranks[0] + 2))
    else:
        ndcgs.append(0)

# Compute final metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit Rate@{k}: {hit_rate:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")
print(f"AUC: {auc:.4f}")

### Hit Rate@10: 0.9920
### NDCG@10: 0.9879
### AUC: 0.9556

In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('pairwise_loss_model.png')
plt.show()

In [ ]:
user_id = 'jay'  # Replace with a valid user identifier as in your mapping
user_idx = user_mapping.get(user_id)

all_items = np.arange(n_items)  
user_tensor = np.array([user_idx] * n_items)  

# Get predicted scores using the scoring network.
predicted_ratings = scoring_net.predict([user_tensor, all_items], verbose=0)

# Flatten and sort predictions to get the top recommendations.
top_items = np.argsort(predicted_ratings.flatten())[::-1][:10]

# Map back the indices to movie IDs (or names). 
# This reverse mapping depends on how item_mapping is defined.
# One way is to create a reverse lookup dictionary:
reverse_item_mapping = {v: k for k, v in item_mapping.items()}
recommended_movie_ids = [reverse_item_mapping[idx] for idx in top_items]

print(f"Recommended movies for user {user_id}: {recommended_movie_ids}")

In [ ]:
user_id = 'jay'  
user_idx = user_mapping.get(user_id)
print("User index:", user_idx)

all_items = np.arange(n_items)

user_seen_items = train_dict.get(user_idx, set())

candidate_items = np.array([item for item in all_items if item not in user_seen_items])

print(f"Number of candidate items for user {user_id}: {len(candidate_items)}")

user_tensor = np.full(candidate_items.shape, user_idx)

predicted_scores = scoring_net.predict([user_tensor, candidate_items], verbose=0)
predicted_scores = predicted_scores.flatten()

top_indices = np.argsort(predicted_scores)[::-1][:10]
top_candidate_items = candidate_items[top_indices]

reverse_item_mapping = {v: k for k, v in item_mapping.items()}
recommended_movie_ids = [reverse_item_mapping[idx] for idx in top_candidate_items]

print(f"Recommended movies for user {user_id} (only from unseen items): {recommended_movie_ids}")

## Model with pointwise loss function (mean squared error) - shows significant bias towards higher rated movies in the reccomendations. 

In [ ]:
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')

user_embedding = Embedding(n_users, embedding_dim, name='user_embedding')(user_input)
item_embedding = Embedding(n_items, embedding_dim, name='item_embedding')(item_input)

user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

gmf_vec = Multiply()([user_vec,item_vec])
concat_vec = Concatenate()([user_vec, item_vec])
mlp = concat_vec
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

pre_output_concatenate = Concatenate()([gmf_vec,mlp])
output = Dense(1, activation='linear', name='output')(pre_output_concatenate)

ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

ncf_model.summary()


In [ ]:
#tf.config.get_visible_devices()

In [ ]:
train_dict = defaultdict(set)

# Assuming your train DataFrame has columns 'user_id' and 'movie_id':
for user, item in zip(train['user_id'], train['movie_id']):
    train_dict[user].add(item)
    

In [ ]:
x_train = [train['user_id'].values, train['movie_id'].values]
y_train = train['rating_val'].values
x_val = [val['user_id'].values, val['movie_id'].values]
y_val = val['rating_val'].values
x_test = [test['user_id'].values, test['movie_id'].values]
y_test = test['rating_val'].values

In [ ]:

print(y_test.shape)

print(y_val.shape)
print(type(x_train))
print(type(x_test))
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)
# Train the model
history = ncf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1,
    callbacks = [early_stop_callback]
)
#uncomment above to train model again, weights can be loaded aswell

In [ ]:
#ncf_model.load_weights('ncf_model.weights.h5')

In [ ]:
ncf_model.save_weights('ncf_model.weights.h5')

In [ ]:
loss, mae = ncf_model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.4f}, Test MAE: {mae:.4f}")

# RMSE
predictions = ncf_model.predict(x_test)
rmse = sqrt(np.mean((predictions.flatten() - y_test) ** 2))
print(f"Test RMSE: {rmse:.4f}")

In [ ]:
print(predictions[1])

In [ ]:
user_id = 'jay'  # Replace with a valid user_id
user_idx = user_mapping.get(user_id)
print(user_idx)
# Predict ratings for all items
all_items = np.arange(n_items)
user_tensor = np.array([user_idx] * n_items)
predicted_ratings = ncf_model.predict([user_tensor, all_items])

# Get top 10 recommendations
top_items = np.argsort(predicted_ratings.flatten())[::-1][:10]
recommended_movie_ids = [list(item_mapping.keys())[list(item_mapping.values()).index(idx)] for idx in top_items]
print(f"Recommended movies for user {user_id}: {recommended_movie_ids}")

In [ ]:
test = test.rename(columns={'rating_val': 'prediction'})

In [ ]:
print(test)

In [ ]:
# all_user_ids = list(user_mapping.keys())       # e.g. original user IDs
# all_user_idxs = list(user_mapping.values())    # integer indices
# BATCH_SIZE = 512
# all_items = np.arange(n_items)

# predictions_list = []
# for i in range(0, len(all_user_idxs), BATCH_SIZE):
#     # Take a chunk of users
#     user_idx_batch = all_user_idxs[i : i + BATCH_SIZE]
#     user_id_batch = all_user_ids[i : i + BATCH_SIZE]

#     # Repeat items for each user in the batch
#     # shape: (#users_in_batch * n_items,)
#     tile_users = np.repeat(user_idx_batch, n_items)
#     tile_items = np.tile(all_items, len(user_idx_batch))

#     # Model predict on that entire chunk
#     batch_preds = ncf_model.predict([tile_users, tile_items])  

#     # Now we map predictions back to (user, item) pairs
#     # We'll build a DataFrame
#     df_chunk = pd.DataFrame({
#         "userID": np.repeat(user_id_batch, n_items),
#         "itemID": tile_items,
#         "prediction": batch_preds
#     })
#     predictions_list.append(df_chunk)

# predictions_df = pd.concat(predictions_list, ignore_index=True)
### 1 HOUR 45 MINS FOR ENTIRE DATASET 

In [ ]:
# print(predictions_df)

In [ ]:
test.rename(
    columns={
        "movie_id": "itemID",
        "user_id": "userID",
        "rating_val": "rating"  
    },
    inplace=True,
)

train.rename(
    columns={
        "movie_id": "itemID",  
        "user_id": "userID"
    },
    inplace=True,
)

In [ ]:
# # Evaluate predictions
# #eval_map = map(test, predictions_df, col_prediction='prediction', k=TOP_K)
# eval_ndcg = ndcg_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# eval_precision = precision_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# eval_recall = recall_at_k(test, predictions_df, col_prediction='prediction', k=TOP_K)
# print("reaches this")
# print(
#     #f"MAP:\t{eval_map:.6f}",
#     f"NDCG:\t{eval_ndcg:.6f}",
#     f"Precision@K:\t{eval_precision:.6f}",
#     f"Recall@K:\t{eval_recall:.6f}",
#     sep='\n'
# )

## COMPLEXITY OF PREDICTING FOR ALL USERS AND ITEMS IS (no_users * no_items) -> VERY LARGE. 
Use negative sampling - take 1 item user interacted with, and 50 or 100 items the user did NOT interact (did not rate). 

# with manual testing for different users, the model basically outputs the highest rated shows and movies that the user has not rated. how to improve on that ? -> tokenization of movie descriptions into the model, another pipeline with the genre. 

In [ ]:
print(train)

In [ ]:
print(test.info())

In [ ]:
all_items_set = set(test['itemID'].unique()) | set(test['itemID'].unique())

In [ ]:
# Parameters
k = 10
hits = []
ndcgs = []
all_true_labels = []
all_pred_scores = []
recommended_items = []
rating_threshold = 8
unique_user_ids = test['userID'].unique()

for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    test_items = test.loc[test['userID'] == user_id & (test['prediction']), 'itemID'].values
    pos_items = set(test_items)  
    limited_pos_items = list(pos_items)[:3] # take only 3, on processor only takes too slow for more
    user_train_items = train_dict[user_id] | set(limited_pos_items)
    possible_negatives = list(all_items_set - user_train_items)

    for pos_item in limited_pos_items:
        neg_sample = random.sample(list(all_items_set - train_dict[user_id] - {pos_item}), 99)
        
        item_batch = [pos_item] + neg_sample
        user_batch = [user_id] * len(item_batch)
        
        scores = ncf_model.predict([np.array(user_batch), np.array(item_batch)])
        scores = np.squeeze(scores)
        
        labels = np.array([1] + [0]*99) # label the highest rated item as positive 
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)
        
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]
        
        top_k_items = sorted_items[:k]
        recommended_items.extend(top_k_items.tolist())
        
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)
        
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)

hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
auc = roc_auc_score(all_true_labels, all_pred_scores)



In [ ]:
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit Rate@{k}: {hit_rate:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")
print(f"ROC AUC: {auc:.4f}")

## Interpretation :
- 45% of users get something relevant to them recommended 
- ndcg : 29% to an ideal ranking system (higher relevance items raise it )


In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Interesting to check the distribution of the recomended movies for each user. 

In [ ]:
item_counts = Counter(recommended_items)
item_counts_df = pd.DataFrame.from_dict(item_counts, orient='index', columns=['count'])
item_counts_df.reset_index(inplace=True)
item_counts_df.rename(columns={'index': 'itemID'}, inplace=True)
item_counts_df.sort_values(by='count', ascending=False, inplace=True)

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(item_counts_df['count'], bins=50, kde=True)
plt.xlabel('Number of Recommendations')
plt.ylabel('Frequency')
plt.title('Distribution of Recommended Movies')
plt.show()

In [ ]:
## heavy bias to the highest rated movies (can also be seen by the manual checks for each user)

In [ ]:
# Hit@10: 0.4800
#NDCG@10: 0.3067

In [ ]:
# The relevant items is in 48 percent of reccomended lists to the user, ndcg of 30 percent means we are 30 percent to an ideal ranking 

## This is a good baseline, however it can be improved, in multiple ways

## 1. Train GMF and MLP models separately, then combine into new model, fine-tune that one to see if improvements (paper suggests this technique gives better results)

In [ ]:
user_gmf_embedding = Embedding(n_users, embedding_dim, name='user_gmf_embedding')(user_input)
item_gmf_embedding = Embedding(n_items, embedding_dim, name='item_gmf_embedding')(item_input)

# Flatten embeddings
user_gmf_vec = Flatten()(user_gmf_embedding)
item_gmf_vec = Flatten()(item_gmf_embedding)

# Element-wise product of user and item embeddings
gmf_vec = Multiply()([user_gmf_vec, item_gmf_vec])

# Output layer
gmf_output = Dense(1, activation='linear', name='gmf_output')(gmf_vec)

# Build and compile GMF model
gmf_model = Model(inputs=[user_input, item_input], outputs=gmf_output)
gmf_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

gmf_model.summary()

In [ ]:
# MLP model - neural network 
user_mlp_embedding = Embedding(n_users, embedding_dim, name='user_mlp_embedding')(user_input)
item_mlp_embedding = Embedding(n_items, embedding_dim, name='item_mlp_embedding')(item_input)

# Flatten embeddings
user_mlp_vec = Flatten()(user_mlp_embedding)
item_mlp_vec = Flatten()(item_mlp_embedding)

# Concatenate embeddings
mlp_vec = Concatenate()([user_mlp_vec, item_mlp_vec])

# Fully connected layers
mlp = mlp_vec
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

# Output layer
mlp_output = Dense(1, activation='linear', name='mlp_output')(mlp)

# Build and compile MLP model
mlp_model = Model(inputs=[user_input, item_input], outputs=mlp_output)
mlp_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [ ]:
x_train = [train['user_id'].values, train['movie_id'].values]
y_train = train['rating_val'].values
x_val = [val['user_id'].values, val['movie_id'].values]
y_val = val['rating_val'].values
x_test = [test['user_id'].values, test['movie_id'].values]
y_test = test['rating_val'].values
# Train GMF model
gmf_history = gmf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1
)

# Train MLP model
mlp_history = mlp_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1
)

In [ ]:
#gmf_model.save_weights('ncf_model.weights.h5')
gmf_model.save_weights('gmf_model.weights.h5')
mlp_model.save_weights('mlp_model.weights.h5')

In [ ]:
alpha = 0.5  

# Weighted combination of GMF and MLP vectors
gmf_weighted = Multiply()([gmf_vec, Lambda(lambda x: x * alpha)(gmf_vec)])
mlp_weighted = Multiply()([mlp, Lambda(lambda x: x * (1 - alpha))(mlp)])

combined = Concatenate()([gmf_weighted, mlp_weighted])

final_output = Dense(1, activation='linear', name='final_output')(combined)

# Build the final NCF model
ncf_combined_model = Model(inputs=[user_input, item_input], outputs=final_output)
ncf_combined_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

ncf_combined_model.summary()

In [ ]:
alphas = [0.25, 0.5, 0.75]
# trained on all 3, balanced one gave best results 

In [ ]:
gmf_model.load_weights('gmf_model.weights.h5')
mlp_model.load_weights('mlp_model.weights.h5')

for layer in gmf_model.layers:
    if layer.name in ncf_combined_model.layers:
        ncf_combined_model.get_layer(layer.name).set_weights(layer.get_weights())

for layer in mlp_model.layers:
    if layer.name in ncf_combined_model.layers:
        ncf_combined_model.get_layer(layer.name).set_weights(layer.get_weights())

# Now, you can train the combined model
ncf_combined_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=5,
    validation_data=(x_val, y_val),
    verbose=1
)

In [ ]:
# Parameters
k = 10
hits = []
ndcgs = []
precisions = []
recalls = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

unique_user_ids = test['user_id'].unique()

# Evaluation Loop
for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # Fetch the test items for the user
    test_items = test.loc[(test['user_id'] == user_id) & (test['rating_val'] >= rating_threshold), 'movie_id'].values
    pos_items = set(test_items)  
    limited_pos_items = list(pos_items)[:3] # take only 3, on processor only takes too slow for all 
    # Build negative set (exclude train_dict[user_id] + the positive items)
    user_train_items = train_dict[user_id] | set(limited_pos_items)
    possible_negatives = list(all_items_set - user_train_items)

    # Iterate over each positive item
    for pos_item in limited_pos_items:
        # Sample 99 negatives for the current positive item
        neg_sample = random.sample(list(all_items_set - train_dict[user_id] - {pos_item}), 99)
        
        # Combine into a batch
        item_batch = [pos_item] + neg_sample
        user_batch = [user_id] * len(item_batch)
        
        # Get predictions
        scores = ncf_combined_model.predict([np.array(user_batch), np.array(item_batch)])
        scores = np.squeeze(scores)
        
        # Binary labels: 1 for positive item, 0 for negatives
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)
        
        # Sort items by score in descending order
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]
        
        # Collect recommended items for distribution analysis
        top_k_items = sorted_items[:k]
        recommended_items.extend(top_k_items.tolist())
        
        # Calculate Hit@K, if positive item in these, its a hit
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)
        
        # Calculate NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)
        
        # Calculate Precision@K
        precision = np.sum(sorted_labels[:k]) / k
        precisions.append(precision)
        
        # Calculate Recall@K
        recall = np.sum(sorted_labels[:k]) / len(limited_pos_items)
        recalls.append(recall)

# Calculate Final Metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)


In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(item_counts_df['count'], bins=50, kde=True)
plt.xlabel('Number of Recommendations')
plt.ylabel('Frequency')
plt.title('Distribution of Recommended Movies')
plt.show()

In [ ]:
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit Rate@{k}: {hit_rate:.4f}")
print(f"NDCG@{k}: {ndcg:.4f}")
print(f"ROC AUC: {auc:.4f}")

In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.savefig('good_curve.png')
plt.show()

In [ ]:
user_id = 'jay'  # Replace with a valid user_id
user_idx = user_mapping.get(user_id)
print(user_idx)
# Predict ratings for all items
all_items = np.arange(n_items)
user_tensor = np.array([user_idx] * n_items)
predicted_ratings = ncf_combined_model.predict([user_tensor, all_items])

# Get top 10 recommendations
top_items = np.argsort(predicted_ratings.flatten())[::-1][:10]
recommended_movie_ids = [list(item_mapping.keys())[list(item_mapping.values()).index(idx)] for idx in top_items]
print(f"Recommended movies for user {user_id}: {recommended_movie_ids}")

MAP:	0.044724
NDCG:	0.183073
FROM THE RECOMENDERS PAPER I USED 

Hit Rate@10: 0.6733
NDCG@10: 0.4019
ROC AUC: 0.8771

In [ ]:
ncf_combined_model.save_weights('ncf_combined_model.weights.h5')

In [ ]:
print(movie_data_df.info())

In [ ]:
print(merged_df.head())

## 2. Improve on the suggested architecture :
- use other information in the dataset: ex. genre, popularity, overview of the movies 
- add layers to the model - batch normalization, dropout, change loss function
- evaluate the model with tenfold cross validation 
- ensemble with tokenization of movie overview 
- add precision@k,AUC,recall@k,mean reciprocal rank (MRR)
- l1-l2 regularization
- autoencoder of user and item 

In [ ]:
movie_data_df = movie_data_df.dropna(subset=['genres', 'popularity','overview'] )

In [ ]:
print(ratings_df)

# turning genres column into one hot encoded columns for each genre. Movies with no genre set should have all zeros in the respective columns. 

In [ ]:
max_tokens = 20000 # vocab size 
sequence_length = 300 # output vector length 

In [ ]:
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movie_data_df['genres'])
genre_labels = mlb.classes_  # Get genre labels from the MultiLabelBinarizer
print(genre_labels)
genre_labels_cleaned = [c.replace(" ", "_") for c in mlb.classes_]

df_genres = pd.DataFrame(genres_encoded, columns=genre_labels_cleaned)
print(genre_labels_cleaned)
# Now the columns will be "Science_Fiction" rather than "Science Fiction"
movie_data_df = pd.concat([movie_data_df, df_genres], axis=1)

# Also keep your final list to use in the loop
genre_labels = genre_labels_cleaned
scaler = MinMaxScaler()
movie_data_df['popularity_scaled'] = scaler.fit_transform(movie_data_df[['popularity']])
#movie_data_df = pd.concat([movie_data_df, genres_encoded_df], axis=1)
vectorizer = TextVectorization(
    max_tokens=max_tokens,
    output_sequence_length=sequence_length,
    output_mode='int'
)

sample_size = 50000  # adapt on a sample large enough so that the vectorization keeps the most of the context data as possible in each review
sample_overviews = movie_data_df['overview'].dropna().sample(n=sample_size, random_state=42)
overview_ds = tf.data.Dataset.from_tensor_slices(sample_overviews)
vectorizer.adapt(overview_ds.batch(512))  # adapt in batches for memory efficiency


In [ ]:
all_overviews = movie_data_df['overview'].fillna("") 

all_overview_ds = tf.data.Dataset.from_tensor_slices(all_overviews).batch(512)

all_overview_ints = []
for batch in all_overview_ds:
    int_batch = vectorizer(batch)  
    all_overview_ints.append(int_batch.numpy())  

all_overview_ints = np.concatenate(all_overview_ints, axis=0)

movie_data_df['overview_ints'] = list(all_overview_ints)

In [ ]:
print(movie_data_df['overview_ints'])

In [ ]:
text_embedding_dim = 32

In [ ]:
# pretrain a model that embeds the texts and pools them 


text_input = Input(shape=(sequence_length,), dtype='int32', name='text_input')

text_emb_layer = Embedding(input_dim=max_tokens, output_dim=text_embedding_dim, name='text_embedding')
embedded_seq = text_emb_layer(text_input)  # shape: (batch, sequence_length, text_embedding_dim)

overview_embedding = GlobalAveragePooling1D()(embedded_seq)  # shape: (batch, text_embedding_dim)

text_model = Model(inputs=text_input, outputs=overview_embedding)
text_model.summary()

In [ ]:
overview_ints_dataset = tf.data.Dataset.from_tensor_slices(all_overview_ints).batch(512)

all_embeddings = []
for batch in overview_ints_dataset:  # first train it on all batches
    text_model(batch,training = True)
for batch in overview_ints_dataset:
    # Pass the batch through text_model
    emb_batch = text_model(batch, training=False)  # shape: (batch_size, text_embedding_dim)
    all_embeddings.append(emb_batch.numpy())

all_embeddings = np.concatenate(all_embeddings, axis=0)  # shape: (num_movies, text_embedding_dim)


In [ ]:
movie_data_df['overview_pre_embed'] = list(all_embeddings)

In [ ]:
movie_data_df = movie_data_df.drop(['genres'],axis=1)

In [ ]:
print(movie_data_df.info())

In [ ]:
print(ratings_df.head())

In [ ]:
#ratings_df['movie_id'] = ratings_df['movie_id'].astype(int)
#movie_data_df['movie_id'] = movie_data_df['movie_id'].astype(int)
movie_data_df = movie_data_df.dropna(subset=['movie_id'])
ratings_df = ratings_df.dropna(subset=['movie_id'])

In [ ]:
#print(list(mlb.classes_))

In [ ]:
#print(genre_labels)

In [ ]:
# Example function to downcast numerics
def downcast_df(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if str(df[col].dtype) == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif str(df[col].dtype) == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [ ]:
print(movie_data_df.info())

In [ ]:
ratings_df = downcast_df(ratings_df)
movie_data_df = downcast_df(movie_data_df)

In [ ]:
genre_columns = [col for col in movie_data_df.columns if col in genre_labels]
print(genre_columns)

In [ ]:
movie_data_df = movie_data_df.drop_duplicates(subset=['movie_id'])

In [ ]:
print(ratings_df.info(memory_usage='deep'))
print(movie_data_df.info(memory_usage='deep'))

In [ ]:
useful_cols = ['movie_id', 'overview_pre_embed','popularity_scaled']+ genre_labels #+ genre_columns, add popularity scaled as well 
movie_data_small = movie_data_df[useful_cols]

merged_df_large = ratings_df.merge(
    movie_data_small,  
    on='movie_id', 
    how='inner'
)


In [ ]:
print(merged_df_large.info())

In [ ]:
merged_df_large = merged_df_large.dropna(subset = genre_labels)

In [ ]:
train1, val1 = train_test_split(merged_df_large, test_size=0.2, random_state=42)
train1, test1 = train_test_split(train1, test_size = 0.2, random_state = 42)

In [ ]:
X_user = train1['user_id'].values
X_item = train1['movie_id'].values
X_text = np.vstack(train1['overview_pre_embed'].values)  # shape (num_samples, text_embedding_dim)
X_popularity = train1['popularity_scaled'].values.reshape(-1, 1)  # shape (num_samples, 1)
X_genres = train1[genre_labels].values  # shape (num_samples, len(genre_columns))

In [ ]:
y = train1['rating_val'].values  

In [ ]:
print(val1.info())

In [ ]:
num_users = X_user.max() + 1
num_items = X_item.max() + 1
num_genres = len(genre_columns)

In [ ]:
print("Data shapes:")
print("X_user:", X_user.shape)
print("X_item:", X_item.shape)
print("X_text:", X_text.shape)
print("X_popularity:", X_popularity.shape)
print("X_genres:", X_genres.shape)
print("y:", y.shape)

In [ ]:
embedding_dim = 32
mlp_layer_sizes = [64, 32, 16]
n_users = 10000
n_items = 5000
num_genres = 19
overview_dim = 32

user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
genre_input = Input(shape=(num_genres,), name='genre_input')
popularity_input = Input(shape=(1,), name='popularity_input')
overview_input = Input(shape=(overview_dim,), name='overview_input')

# ---------------------------
# 2) Embeddings (Reused)
# ---------------------------
user_embedding = Embedding(n_users, embedding_dim, name='user_embedding')
item_embedding = Embedding(n_items, embedding_dim, name='item_embedding')

user_vec = Flatten()(user_embedding(user_input))
item_vec = Flatten()(item_embedding(item_input))

# ---------------------------
# 3) GMF path (element-wise product)
# ---------------------------
gmf_vec = Multiply(name='gmf_interaction')([user_vec, item_vec])

# ---------------------------
# 4) MLP path (concat user_vec, item_vec & item features)
# ---------------------------
# Reduce overview dimension
#overview_reduced = Dense(32, activation='relu', name='overview_reduced')(overview_input)

item_features = Concatenate(name='item_features_concat')([
    item_vec,
    genre_input,
    popularity_input,
    overview_input
])

mlp_input = Concatenate(name='mlp_concat')([
    user_vec,
    item_features
])

mlp = mlp_input
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

mlp_vec = mlp

# ---------------------------
# 5) Final: Concatenate GMF & MLP + Output
# ---------------------------
final_concat = Concatenate(name='final_concat')([gmf_vec, mlp_vec])
output = Dense(1, activation='linear', name='output')(final_concat)

ncf_model = Model(
    inputs=[
        user_input,
        item_input,
        genre_input,
        popularity_input,
        overview_input
    ],
    outputs=output
)

# ---------------------------
# 6) Compile & Summary
# ---------------------------
ncf_model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)

ncf_model.summary()

In [ ]:
X_train = [
    X_user,        # user_input
    X_item,        # item_input
    X_text,        # text_input
    X_popularity,  # popularity_input
    X_genres       # genre_input
]

In [ ]:
# model.fit(
#     x= X_train,
#     y=y,
#     batch_size=256,
#     epochs=5,
#     #validation_split=0.1
#     validation_data = [val1]
# )

In [ ]:

# 1) user_id: shape (num_samples,)
user_ids_train = train1['user_id'].values  

# 2) movie_id: shape (num_samples,)
item_ids_train = train1['movie_id'].values

# 3) genre vector: shape (num_samples, num_genres)
genre_train = train1[genre_labels].values  

# 4) popularity: shape (num_samples, 1)
popularity_train = train1['popularity_scaled'].values.reshape(-1, 1)

# 5) overview: shape (num_samples, overview_dim)
overview_train = np.vstack(train1['overview_pre_embed'].values)

# y: shape (num_samples,)
y_train = train1['rating_val'].values

x_train = [
    user_ids_train,       
    item_ids_train,       
    genre_train,          
    popularity_train,     
    overview_train        
]

user_ids_val = val1['user_id'].values
item_ids_val = val1['movie_id'].values
genre_val = val1[genre_labels].values
popularity_val = val1['popularity_scaled'].values.reshape(-1, 1)
overview_val = np.vstack(val1['overview_pre_embed'].values)
y_val = val1['rating_val'].values

x_val = [
    user_ids_val,
    item_ids_val,
    genre_val,
    popularity_val,
    overview_val
]


In [ ]:
print(np.isnan(y_train).any())

In [ ]:
print(np.isnan(overview_train).any())

In [ ]:
print(np.isnan(popularity_train).any())

In [ ]:
print(np.isnan(genre_train).any())

In [ ]:
#big_model.load_weights('ncf_combined_model.weights.h5')
# try without loading the pretrained first 

In [ ]:
print(train.info())

In [ ]:
print(train1.info())

In [ ]:
history = ncf_model.fit(
    x=x_train,
    y=y_train,
    batch_size=256,
    epochs=10,
    validation_data=(x_val, y_val),
    verbose=1
)

In [ ]:
#ncf_model.load_weights('ncf_model_big.weights.h5')

In [ ]:
ncf_model.save_weights('ncf_model_big.weights.h5')

In [ ]:
train_dict1 = defaultdict(set)

# Assuming your train DataFrame has columns 'user_id' and 'movie_id':
for user, item in zip(train1['user_id'], train1['movie_id']):
    train_dict1[user].add(item)
    

In [ ]:
all_items_set = set(train1['movie_id'].unique()) | set(test1['movie_id'].unique())

In [ ]:
item_overview_map = dict(zip(movie_data_df['movie_id'], movie_data_df['overview_pre_embed']))

In [ ]:
item_genre_map = {}
item_popularity_map = {}
item_overview_map = {}
test_item_ids = set(test1['movie_id'].unique())

sub_df = merged_df_large[merged_df_large['movie_id'].isin(test_item_ids)]

for idx, row in tqdm(sub_df.iterrows(), 
                     total=len(sub_df), 
                     desc="Processing test items"):
    item_id = row["movie_id"]
    genre_vec = np.array([row[g] for g in genre_labels], dtype=np.float32)

    # Suppose you have genre columns g1...g19 in your DF
    # Create the genre vector:
    # If you stored them as separate columns, collect them:
    
    # Popularity
    popularity_val = row.popularity_scaled  # or getattr(row, 'popularity_scaled')

    # Overview pre-embedding (if stored as a list in the DF, you might do something else)
    # If you stored them as an actual Python list or array in the DataFrame, just reference it
    overview_vec = np.array(row.overview_pre_embed, dtype=np.float32)

    item_genre_map[item_id] = genre_vec
    item_popularity_map[item_id] = popularity_val
    item_overview_map[item_id] = overview_vec


In [ ]:
print(test1[genre_labels].isna().sum())

In [ ]:
print(test1.head())

In [ ]:
#print(movie_data_df[genre_labels].isna().sum())

In [ ]:

k = 10
hits = []
ndcgs = []
precisions = []
recalls = []
all_true_labels = []
all_pred_scores = []
recommended_items = []

unique_user_ids = test1['user_id'].unique()

all_items_set = set(movie_data_df['movie_id'])  

for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # 1) Fetch the test (positive) items for this user - he rated (interacted)
    test_items = test1.loc[test1['user_id'] == user_id, 'movie_id'].values
    pos_items = set(test_items)

    # If you only want to evaluate, say, 3 positives:
    limited_pos_items = list(pos_items)[:3]

    # 2) Retrieve training items for user (to exclude them in negative sampling)
    #    Assuming you have a dict: train_dict[user_id] = set of items user has interacted with in train
    user_train_items = train_dict1[user_id] | set(limited_pos_items)
    
    # 3) Negative sampling
    possible_negatives = all_items_set - user_train_items

    for pos_item in limited_pos_items:
        # Sample 99 negatives
        neg_sample = random.sample(list(possible_negatives - {pos_item}), 99)

        # item_batch are the actual item IDs
        item_batch = [pos_item] + neg_sample

        # user_batch is the user repeated
        user_batch = [user_id] * len(item_batch)
        item_batch_np = np.array(item_batch, dtype=np.int32)
        item_batch = [int(i) for i in item_batch]
        # 2) genre_batch_np
        genre_batch = [item_genre_map[i] for i in item_batch]
        genre_batch_np = np.array(genre_batch, dtype=np.float32)
        
        pop_batch = [item_popularity_map[i] for i in item_batch]
        popularity_batch_np = np.array(pop_batch, dtype=np.float32).reshape(-1, 1)
        
        # 4) overview_batch_np
        ov_batch = [item_overview_map[i] for i in item_batch]
        overview_batch_np = np.array(ov_batch, dtype=np.float32)
        # 4) Build item_emb_batch by looking up each item ID in the embedding map
        #item_emb_batch = [item_overview_map[item] for item in item_batch]
        # item_emb_batch is now a list of arrays (each shape = (text_embedding_dim,))

        # Convert to NumPy
        user_batch_np = np.array(user_batch, dtype=np.int32)         # shape: (batch_size,)
        #item_emb_batch_np = np.array(item_emb_batch, dtype=np.float32)  # shape: (batch_size, text_embedding_dim)

        # 5) Predict
        scores = ncf_model.predict([
        user_batch_np,
        item_batch_np,
        genre_batch_np,
        popularity_batch_np,
        overview_batch_np
        ], verbose=0)
        scores = np.squeeze(scores) 

        # 6) Prepare labels (1 positive, 99 negatives)
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)

        # 7) Sort items by predicted score (descending)
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]

        # 8) Evaluate metrics
        # Hit@K
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)

        # NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))  
        else:
            ndcgs.append(0)

        # Precision@K
        precision = np.sum(sorted_labels[:k]) / k
        precisions.append(precision)

        # Recall@K
        recall = np.sum(sorted_labels[:k]) / len(limited_pos_items)
        recalls.append(recall)

# 9) Compute final metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
precision_at_k = np.mean(precisions)
recall_at_k = np.mean(recalls)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit@{k}: {hit_rate}")
print(f"NDCG@{k}: {ndcg}")
print(f"Precision@{k}: {precision_at_k}")
print(f"Recall@{k}: {recall_at_k}")
print(f"AUC: {auc}")

In [ ]:
print("Any NaN in user_batch_np?", np.isnan(user_batch_np).any())
print("Any NaN in item_batch_np?", np.isnan(item_batch_np).any())
print("Any NaN in genre_batch_np?", np.isnan(genre_batch_np).any())
print("Any NaN in popularity_batch_np?", np.isnan(popularity_batch_np).any())
print("Any NaN in overview_batch_np?", np.isnan(overview_batch_np).any())

In [ ]:
fpr, tpr, thresholds = roc_curve(all_true_labels, all_pred_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
print(test1.info())

### 3. Pretrained GMF,MLP and overview based model combined together. 

In [ ]:
user_embedding_dim = 32


In [ ]:
user_input = Input(shape=(1,), dtype='int32', name='user_input')
text_input = Input(shape=(sequence_length,), dtype='int32', name='text_input')

# ---------------------------
# 2) User Embedding
# ---------------------------
user_embedding_layer = Embedding(
    input_dim=n_users, 
    output_dim=user_embedding_dim, 
    name='user_embedding'
)
user_emb = user_embedding_layer(user_input)  # shape (None, 1, user_embedding_dim)
user_vec = Flatten()(user_emb)               # shape (None, user_embedding_dim)

# ---------------------------
# 3) Text Embedding
# ---------------------------
text_embedding_layer = Embedding(
    input_dim=max_tokens, 
    output_dim=text_embedding_dim, 
    name='text_embedding'
)
text_embedded_seq = text_embedding_layer(text_input)  
# shape (None, sequence_length, text_embedding_dim)

# Global average pooling to get a single vector
text_vec = GlobalAveragePooling1D()(text_embedded_seq)
# shape (None, text_embedding_dim)

# ---------------------------
# 4) Combine User + Text
# ---------------------------
# Simple approach: concatenate user vector & text vector, then MLP
combined_vec = Concatenate()([user_vec, text_vec])  # shape (None, user_embedding_dim + text_embedding_dim)

x = Dense(64, activation='relu')(combined_vec)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.2)(x)

# Final rating output
output = Dense(1, activation='linear', name='rating_output')(x)

# ---------------------------
# 5) Build & Compile
# ---------------------------
text_only_cf_model = Model(
    inputs=[user_input, text_input],
    outputs=output
)

text_only_cf_model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

text_only_cf_model.summary()

In [ ]:
useful_cols = ['movie_id', 'overview']
movie_data_small = movie_data_df[useful_cols]
merged_df = ratings_df.merge(
    movie_data_small,  
    on='movie_id', 
    how='inner'
)

In [ ]:
print(merged_df.info())

In [ ]:
sample_size = 150000
df_sample = merged_df.sample(n=sample_size, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=max_tokens, 
                      oov_token="<UNK>")

In [ ]:
overview_list = list(tqdm(df_sample['overview'], desc="Collecting sample texts"))

# Fit tokenizer
tokenizer.fit_on_texts(overview_list)

In [ ]:
all_overview_sequences = tokenizer.texts_to_sequences(tqdm(df_sample['overview'], 
                                                          desc="Converting all texts"))

In [ ]:
print(df_sample.info())

In [ ]:
text_input_data = pad_sequences(all_overview_sequences, 
                                maxlen=sequence_length, 
                                padding='post', 
                                truncating='post')

In [ ]:
print(text_input_data)

In [ ]:
unique_users = df_sample['user_id'].unique()
user2idx = {old_id: new_id for new_id, old_id in enumerate(unique_users)}
df_sample['user_id_mapped'] = df_sample['user_id'].map(user2idx)
n_users = len(unique_users)

In [ ]:
ratings = df_sample['rating_val'].values

In [ ]:
train2, val2 = train_test_split(df_sample, test_size=0.2, random_state=42)
train2, test2 = train_test_split(train2, test_size = 0.2, random_state = 42)

In [ ]:
all_overview_sequences2 = tokenizer.texts_to_sequences(tqdm(train2['overview'], 
                                                          desc="Converting all train texts"))
text_input_data = pad_sequences(all_overview_sequences2, 
                                maxlen=sequence_length, 
                                padding='post', 
                                truncating='post')

In [ ]:
all_overview_sequences2_val = tokenizer.texts_to_sequences(tqdm(val2['overview'], 
                                                          desc="Converting all train texts"))
text_input_data_val = pad_sequences(all_overview_sequences2_val, 
                                maxlen=sequence_length, 
                                padding='post', 
                                truncating='post')

In [ ]:
all_overview_sequences2_test = tokenizer.texts_to_sequences(tqdm(test2['overview'], 
                                                          desc="Converting all train texts"))
text_input_data_test = pad_sequences(all_overview_sequences2_val, 
                                maxlen=sequence_length, 
                                padding='post', 
                                truncating='post')

In [ ]:
train2['overview_length'] = train2['overview'].apply(len)

avg_length = train2['overview_length'].mean()
min_length = train2['overview_length'].min()
max_length = train2['overview_length'].max()
train2.drop(['overview_length'],axis = 1)
print(f"Average Overview Length: {avg_length:.2f}")
print(f"Minimum Overview Length: {min_length}")
print(f"Maximum Overview Length: {max_length}")


In [ ]:
text_embedding_dim = 32
sequence_length = 32 # 1000 with untokenized 
text_input = Input(shape=(sequence_length,), dtype='int32', name='text_input')

text_emb_layer = Embedding(input_dim=max_tokens, output_dim=text_embedding_dim, name='text_embedding')
embedded_seq = text_emb_layer(text_input)  # (batch, sequence_length, text_embedding_dim)

# Global average pooling to get fixed-size vector (32)
text_embedding = GlobalAveragePooling1D()(embedded_seq)  # (batch, 32)

# Define text embedding model
text_model = Model(inputs=text_input, outputs=text_embedding)

In [ ]:
train_user_input = train2['user_id_mapped'].values
train_texts = train2['overview'].values
train_y = train2['rating_val'].values

# Validation sets
val_user_input = val2['user_id_mapped'].values
val_texts = train2['overview'].values
val_y = val2['rating_val'].values

# Test sets
test_user_input = test2['user_id_mapped'].values
test_texts = train2['overview'].values
test_y = test2['rating_val'].values

In [ ]:
user_input = Input(shape=(1,), name='user_input')

# User embedding layer (same as before)
user_embedding = Embedding(n_users, embedding_dim, name='user_embedding')(user_input)
user_vec = Flatten()(user_embedding)  # (batch, 32)

# Get text embedding from the pre-trained text model
item_embedding = text_model(text_input)  # (batch, 32)
text_embedding_1000 = text_model(text_input)            # shape: (batch, 1000)
item_embedding = Dense(32, activation='relu')(
    text_embedding_1000
)  
# GMF component
gmf_vec = Multiply()([user_vec, item_embedding])

# MLP component
concat_vec = Concatenate()([user_vec, item_embedding])  # (batch, 64)
mlp = concat_vec
for size in mlp_layer_sizes:
    mlp = Dense(size, activation='relu')(mlp)
    mlp = Dropout(0.2)(mlp)

# Final prediction layer
pre_output_concatenate = Concatenate()([gmf_vec, mlp])  # (batch, 48)
output = Dense(1, activation='linear', name='output')(pre_output_concatenate)

# Build the full NCF model
ncf_text_model = Model(inputs=[user_input, text_input], outputs=output)
ncf_text_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

ncf_text_model.summary()

In [ ]:
print(train2.info())

In [ ]:
print(test2.shape)

In [ ]:
ncf_text_model.fit([train_user_input, text_input_data], train_y, validation_data=([val_user_input, text_input_data_val], val_y), epochs=10, batch_size=32)

In [ ]:
train_dict = defaultdict(set)

# Assuming your train DataFrame has columns 'user_id' and 'movie_id':
for user, item in zip(train2['user_id'], train2['movie_id']):
    train_dict[user].add(item)
    

In [ ]:
all_items_set = set(train2['movie_id'].unique()) | set(test2['movie_id'].unique())

In [ ]:
print(test2.info())

In [ ]:
item2index_all = {}
for i, row in df_sample.iterrows():
    item_id = row['movie_id']
    item2index[item_id] = i

In [ ]:
test2 = test2.reset_index(drop=True)   # Now test2.index runs 0..29999
item2index = {
    test2.loc[i, 'movie_id']: i
    for i in range(len(test2))
}

In [ ]:

k = 10
hits = []
ndcgs = []
all_true_labels = []
all_pred_scores = []
recommended_items = []
rating_threshold = 8
unique_user_ids = test2['user_id'].unique()

for user_id in tqdm(unique_user_ids[:100], desc="Processing users"):
    # Fetch positive items from the test set
    test_items = test2.loc[
        (test2['user_id'] == user_id) & (test2['rating_val'] >= rating_threshold),
        'movie_id'
    ].values
    pos_items = set(test_items)
    limited_pos_items = list(pos_items)[:3]

    user_train_items = train_dict[user_id] | set(limited_pos_items)
    possible_negatives = list(all_items_set - user_train_items)

    # Iterate over each positive item
    for pos_item in limited_pos_items:
        # Sample 99 negatives
        neg_sample = random.sample(
            list(all_items_set - train_dict[user_id] - {pos_item}),
            99
        )
        
        # Combine into a batch
        item_batch = [pos_item] + neg_sample  # length=100
        mapped_indices = [item2index_all[item_id] for item_id in item_batch]
        item_batch_texts = text_input_data_test[mapped_indices]  
        user_batch = [user_id] * len(item_batch)

        # Reshape user batch if needed:
        user_batch_array = np.array(user_batch).reshape(-1, 1)

        # Get text data for items
        #item_batch_texts = text_input_data_test[item_batch]  # shape: (100, sequence_length)
        
        # Get predictions from your text-based NCF model
        scores = ncf_text_model.predict([user_batch_array, item_batch_texts])
        scores = np.squeeze(scores)  # (100,)

        # Prepare binary labels (1 for pos_item, 0 for negative items)
        labels = np.array([1] + [0]*99)
        all_true_labels.extend(labels)
        all_pred_scores.extend(scores)

        # Sort items by score (descending)
        sorted_indices = np.argsort(scores)[::-1]
        sorted_labels = labels[sorted_indices]
        sorted_items = np.array(item_batch)[sorted_indices]

        # Grab top-K
        top_k_items = sorted_items[:k]
        recommended_items.extend(top_k_items.tolist())

        # HIT@K
        if 1 in sorted_labels[:k]:
            hits.append(1)
        else:
            hits.append(0)

        # NDCG@K
        rank = np.where(sorted_labels[:k] == 1)[0]
        if len(rank) > 0:
            ndcgs.append(1 / np.log2(rank[0] + 2))
        else:
            ndcgs.append(0)

# Compute final metrics
hit_rate = np.mean(hits)
ndcg = np.mean(ndcgs)
auc = roc_auc_score(all_true_labels, all_pred_scores)

print(f"Hit@{k}: {hit_rate}, NDCG@{k}: {ndcg}, AUC: {auc}")